   # 🧒📝 CommonLit Readability Prize-EDA & Model 📝🧒 


## Objective

In this competition, we're predicting the reading ease of excerpts from literature. We've provided excerpts from several time periods and a wide range of reading ease scores. Note that the test set includes a slightly larger proportion of modern texts (the type of texts we want to generalize to) than the training set.

Also note that while licensing information is provided for the public test set (because the associated excerpts are available for display / use), the hidden private test set includes only blank license / legal information.

### Files

* train.csv - the training set
* test.csv - the test set
* sample_submission.csv - a sample submission file in the correct format

### Columns

* id - unique ID for excerpt
* url_legal - URL of source - this is blank in the test set.
* license - license of source material - this is blank in the test set.
* excerpt - text to predict reading ease of
* target - reading ease
* standard_error - measure of spread of scores among multiple raters for each excerpt. Not included for test data.


### Evaluation Metric :- 

Submissions are scored on the root mean squared error ( RMSE )

### Submission File :- 


 #### id,target
 eaf8e7355,0.0</br>
 60ecc9777,0.5</br>
 c0f722661,-2.0</br>
    


In [ ]:
!pip install sweetviz
!pip install klib
!python3 -m spacy download en_core_web_sm


In [ ]:
import os
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
import klib



## Data Vizualisation library

import plotly as py
import plotly.graph_objs as go
import plotly.figure_factory as ff
import plotly.express as px
from plotly.subplots import make_subplots
from plotly.offline import iplot
from plotly.offline import iplot



In [ ]:
import configparser
import re
import spacy
from pandas.io.json import json_normalize
import os
import string

from nltk.tokenize import word_tokenize
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
nltk.download('punkt')
nltk.download('stopwords')
pd.set_option('max_colwidth',900)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from scipy.sparse import hstack
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.cluster import MiniBatchKMeans
import warnings
warnings.filterwarnings('ignore')
import urllib        #for url stuff

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import text
from IPython.display import display
from tqdm import tqdm
from collections import Counter
import ast

from sklearn.feature_extraction.text import CountVectorizer
from textblob import TextBlob
import scipy.stats as stats

import seaborn as sns
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA, TruncatedSVD
import matplotlib.patches as mpatches
import time

import seaborn as sns #for making plots
import matplotlib.pyplot as plt # for plotting
import os  # for os commands
plt.rcParams['figure.figsize'] = [16, 10]
%matplotlib inline


import gensim
from gensim import corpora, models, similarities
import logging
import tempfile
from nltk.corpus import stopwords
from string import punctuation
from collections import OrderedDict

from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.manifold import TSNE


from bokeh.plotting import figure, output_file, show
from bokeh.models import Label
from bokeh.io import output_notebook
output_notebook()

In [ ]:
from nltk.corpus import stopwords
from wordcloud import WordCloud,STOPWORDS
from sklearn.feature_extraction.text import CountVectorizer as CV
nltk.download('stopwords')

In [ ]:
df_train = pd.read_csv("../input/commonlitreadabilityprize/train.csv")
df_test = pd.read_csv("../input/commonlitreadabilityprize/test.csv")

In [ ]:
df_train.head()

In [ ]:
df_train.describe()

# Data Profiling

# visualization of the number and frequency of categorical features

In [ ]:
klib.cat_plot(df_train) # returns a visualization of the number and frequency of categorical features

### Observation :- 
- "CC-BY-4.0" dominates the more licence catagory , Licence field is highly catagorical than others  

In [ ]:
klib.corr_mat(df_train) # returns a color-encoded correlation matrix

# A color-encoded heatmap, ideal for correlations

In [ ]:
klib.corr_plot(df_train) 

# A distribution plot for every numeric feature

In [ ]:
klib.dist_plot(df_train) # returns 

### Observation :- 
- Target field is notmally distributed , We can choose the appropirate sampling strategy
- Std Error field is bit left sqewed  </br></br>

# Below Plot containing information about missing values

In [ ]:
klib.missingval_plot(df_train) 

### Observation :- 

- url_legal & license have lot of missing values and the missing pattern matches with each other

In [ ]:
import sweetviz as sv
#analyzing the dataset
train_report = sv.analyze(df_train)
#display the report
train_report.show_html()

In [ ]:
train_report.show_notebook()

# Character Length Distribution - Field Expert

In [ ]:
char_freq_count = df_train['excerpt'].str.len()

fig = make_subplots(rows=1, cols=1)

fig.add_trace(
    go.Histogram(x=list(char_freq_count), name='Excerpt Text'),
    row=1, 
    col=1
)

fig.update_layout(title_text="Excerpt Text Character Count")
iplot(fig)

# Unique word count - Field - excerpt

In [ ]:
unq_word_count = df_train['excerpt'].apply(lambda x: len(set(str(x).split()))).to_list()

fig = ff.create_distplot([unq_word_count], ['Excerpt Text'])
fig.update_layout(title_text="Unique Word Count Distribution")
iplot(fig)

In [ ]:
def clean_tweet(text):
  text = re.sub("RT @[\w]*:","",text)
  text = re.sub("@[\w]*","",text)
  text = re.sub("https?://[A-Za-z0-9./]*","",text)
  text = re.sub("\n","",text)
  text = ''.join([x for x in text if x in string.printable])
  return text

def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)


def remove_punctuations(text):
    punctuations = '@#!?+&*[]-%.:/();$=><|{}^' + "'`"
    
    for p in punctuations:
        text = text.replace(p, f' {p} ')

    text = text.replace('...', ' ... ')
    
    if '...' not in text:
        text = text.replace('..', ' ... ')
    
    return text

In [ ]:
abbreviations = {
    "$" : " dollar ",
    "€" : " euro ",
    "4ao" : "for adults only",
    "a.m" : "before midday",
    "a3" : "anytime anywhere anyplace",
    "aamof" : "as a matter of fact",
    "acct" : "account",
    "adih" : "another day in hell",
    "afaic" : "as far as i am concerned",
    "afaict" : "as far as i can tell",
    "afaik" : "as far as i know",
    "afair" : "as far as i remember",
    "afk" : "away from keyboard",
    "app" : "application",
    "approx" : "approximately",
    "apps" : "applications",
    "asap" : "as soon as possible",
    "asl" : "age, sex, location",
    "atk" : "at the keyboard",
    "ave." : "avenue",
    "aymm" : "are you my mother",
    "ayor" : "at your own risk", 
    "b&b" : "bed and breakfast",
    "b+b" : "bed and breakfast",
    "b.c" : "before christ",
    "b2b" : "business to business",
    "b2c" : "business to customer",
    "b4" : "before",
    "b4n" : "bye for now",
    "b@u" : "back at you",
    "bae" : "before anyone else",
    "bak" : "back at keyboard",
    "bbbg" : "bye bye be good",
    "bbc" : "british broadcasting corporation",
    "bbias" : "be back in a second",
    "bbl" : "be back later",
    "bbs" : "be back soon",
    "be4" : "before",
    "bfn" : "bye for now",
    "blvd" : "boulevard",
    "bout" : "about",
    "brb" : "be right back",
    "bros" : "brothers",
    "brt" : "be right there",
    "bsaaw" : "big smile and a wink",
    "btw" : "by the way",
    "bwl" : "bursting with laughter",
    "c/o" : "care of",
    "cet" : "central european time",
    "cf" : "compare",
    "cia" : "central intelligence agency",
    "csl" : "can not stop laughing",
    "cu" : "see you",
    "cul8r" : "see you later",
    "cv" : "curriculum vitae",
    "cwot" : "complete waste of time",
    "cya" : "see you",
    "cyt" : "see you tomorrow",
    "dae" : "does anyone else",
    "dbmib" : "do not bother me i am busy",
    "diy" : "do it yourself",
    "dm" : "direct message",
    "dwh" : "during work hours",
    "e123" : "easy as one two three",
    "eet" : "eastern european time",
    "eg" : "example",
    "embm" : "early morning business meeting",
    "encl" : "enclosed",
    "encl." : "enclosed",
    "etc" : "and so on",
    "faq" : "frequently asked questions",
    "fawc" : "for anyone who cares",
    "fb" : "facebook",
    "fc" : "fingers crossed",
    "fig" : "figure",
    "fimh" : "forever in my heart", 
    "ft." : "feet",
    "ft" : "featuring",
    "ftl" : "for the loss",
    "ftw" : "for the win",
    "fwiw" : "for what it is worth",
    "fyi" : "for your information",
    "g9" : "genius",
    "gahoy" : "get a hold of yourself",
    "gal" : "get a life",
    "gcse" : "general certificate of secondary education",
    "gfn" : "gone for now",
    "gg" : "good game",
    "gl" : "good luck",
    "glhf" : "good luck have fun",
    "gmt" : "greenwich mean time",
    "gmta" : "great minds think alike",
    "gn" : "good night",
    "g.o.a.t" : "greatest of all time",
    "goat" : "greatest of all time",
    "goi" : "get over it",
    "gps" : "global positioning system",
    "gr8" : "great",
    "gratz" : "congratulations",
    "gyal" : "girl",
    "h&c" : "hot and cold",
    "hp" : "horsepower",
    "hr" : "hour",
    "hrh" : "his royal highness",
    "ht" : "height",
    "ibrb" : "i will be right back",
    "ic" : "i see",
    "icq" : "i seek you",
    "icymi" : "in case you missed it",
    "idc" : "i do not care",
    "idgadf" : "i do not give a damn fuck",
    "idgaf" : "i do not give a fuck",
    "idk" : "i do not know",
    "ie" : "that is",
    "i.e" : "that is",
    "ifyp" : "i feel your pain",
    "IG" : "instagram",
    "iirc" : "if i remember correctly",
    "ilu" : "i love you",
    "ily" : "i love you",
    "imho" : "in my humble opinion",
    "imo" : "in my opinion",
    "imu" : "i miss you",
    "iow" : "in other words",
    "irl" : "in real life",
    "j4f" : "just for fun",
    "jic" : "just in case",
    "jk" : "just kidding",
    "jsyk" : "just so you know",
    "l8r" : "later",
    "lb" : "pound",
    "lbs" : "pounds",
    "ldr" : "long distance relationship",
    "lmao" : "laugh my ass off",
    "lmfao" : "laugh my fucking ass off",
    "lol" : "laughing out loud",
    "ltd" : "limited",
    "ltns" : "long time no see",
    "m8" : "mate",
    "mf" : "motherfucker",
    "mfs" : "motherfuckers",
    "mfw" : "my face when",
    "mofo" : "motherfucker",
    "mph" : "miles per hour",
    "mr" : "mister",
    "mrw" : "my reaction when",
    "ms" : "miss",
    "mte" : "my thoughts exactly",
    "nagi" : "not a good idea",
    "nbc" : "national broadcasting company",
    "nbd" : "not big deal",
    "nfs" : "not for sale",
    "ngl" : "not going to lie",
    "nhs" : "national health service",
    "nrn" : "no reply necessary",
    "nsfl" : "not safe for life",
    "nsfw" : "not safe for work",
    "nth" : "nice to have",
    "nvr" : "never",
    "nyc" : "new york city",
    "oc" : "original content",
    "og" : "original",
    "ohp" : "overhead projector",
    "oic" : "oh i see",
    "omdb" : "over my dead body",
    "omg" : "oh my god",
    "omw" : "on my way",
    "p.a" : "per annum",
    "p.m" : "after midday",
    "pm" : "prime minister",
    "poc" : "people of color",
    "pov" : "point of view",
    "pp" : "pages",
    "ppl" : "people",
    "prw" : "parents are watching",
    "ps" : "postscript",
    "pt" : "point",
    "ptb" : "please text back",
    "pto" : "please turn over",
    "qpsa" : "what happens", #"que pasa",
    "ratchet" : "rude",
    "rbtl" : "read between the lines",
    "rlrt" : "real life retweet", 
    "rofl" : "rolling on the floor laughing",
    "roflol" : "rolling on the floor laughing out loud",
    "rotflmao" : "rolling on the floor laughing my ass off",
    "rt" : "retweet",
    "ruok" : "are you ok",
    "sfw" : "safe for work",
    "sk8" : "skate",
    "smh" : "shake my head",
    "sq" : "square",
    "srsly" : "seriously", 
    "ssdd" : "same stuff different day",
    "tbh" : "to be honest",
    "tbs" : "tablespooful",
    "tbsp" : "tablespooful",
    "tfw" : "that feeling when",
    "thks" : "thank you",
    "tho" : "though",
    "thx" : "thank you",
    "tia" : "thanks in advance",
    "til" : "today i learned",
    "tl;dr" : "too long i did not read",
    "tldr" : "too long i did not read",
    "tmb" : "tweet me back",
    "tntl" : "trying not to laugh",
    "ttyl" : "talk to you later",
    "u" : "you",
    "u2" : "you too",
    "u4e" : "yours for ever",
    "utc" : "coordinated universal time",
    "w/" : "with",
    "w/o" : "without",
    "w8" : "wait",
    "wassup" : "what is up",
    "wb" : "welcome back",
    "wtf" : "what the fuck",
    "wtg" : "way to go",
    "wtpa" : "where the party at",
    "wuf" : "where are you from",
    "wuzup" : "what is up",
    "wywh" : "wish you were here",
    "yd" : "yard",
    "ygtr" : "you got that right",
    "ynk" : "you never know",
    "zzz" : "sleeping bored and tired"
}

In [ ]:
def convert_abbrev(word):
    return abbreviations[word.lower()] if word.lower() in abbreviations.keys() else word

In [ ]:
def convert_abbrev_in_text(text):
    tokens = word_tokenize(text)
    tokens = [convert_abbrev(word) for word in tokens]
    text = ' '.join(tokens)
    return text

In [ ]:
df_train['excerpt']

In [ ]:
df_train['excerpt'] = df_train['excerpt'].apply(lambda x :clean_tweet(x))
df_train['excerpt'] = df_train['excerpt'].apply(lambda x :remove_emoji(x))
df_train['excerpt'] = df_train['excerpt'].apply(lambda x :remove_punctuations(x))
df_train['excerpt']= df_train['excerpt'].apply(lambda x :convert_abbrev_in_text(x))

In [ ]:
nlp = spacy.load('en_core_web_sm')

In [ ]:
df_train['entity'] = df_train['excerpt'].apply(lambda x:[(ent.text,ent.label_) if (not ent.text.startswith('#')) else "" for ent in nlp(x).ents])

### Let see sentiment in the text ( Bcoz Children 👶  gonna read this content )

In [ ]:
sid = SentimentIntensityAnalyzer()
df_train['sentiment'] = df_train['excerpt'].apply(lambda x:sid.polarity_scores(x))

In [ ]:
df_train.head(3)

In [ ]:
df_sentiment = json_normalize(df_train['sentiment'])
df_sentiment.columns = ['Sentiment_'+x for x in df_sentiment.columns]
df_train = pd.concat([df_train,df_sentiment],axis=1)

In [ ]:
def sentiment_code(Sentiment_neg,Sentiment_neu,Sentiment_pos):
    
    sc_cd=0
    
    if Sentiment_neg >=0.33 :
        
        sc_cd=2
        
    if Sentiment_neu >=0.33 :
        
        sc_cd=0
        
    if Sentiment_pos >= 0.33:
        
        sc_cd=1
        
    return sc_cd

In [ ]:
df_train['sentiment_ctg']=np.vectorize(sentiment_code)(df_train['Sentiment_neg'],df_train['Sentiment_neu'],df_train['Sentiment_pos'])

In [ ]:
df_train['excerpt_len']=df_train['excerpt'].apply(lambda x:len(x))

# Text length vs Sentiments

In [ ]:
plt.rcParams['figure.figsize'] = (18.0, 6.0)
bins = 150
plt.hist(df_train[df_train['sentiment_ctg'] == 0]['excerpt_len'], alpha = 0.6, bins=bins, label='Neutral')
plt.hist(df_train[df_train['sentiment_ctg'] == 1]['excerpt_len'], alpha = 0.8, bins=bins, label='Positive')
plt.hist(df_train[df_train['sentiment_ctg'] == 2]['excerpt_len'], alpha = 0.4, bins=bins, label='Negative')
plt.xlabel('length')
plt.ylabel('numbers')
plt.legend(loc='upper right')
#plt.xlim(0,150)
plt.grid()
plt.show()

### Observation :- 

- Text is neutral for kids to read

# Please upvote this kernal to share more quality content

![Upvote!](https://img.shields.io/badge/Upvote-If%20you%20like%20my%20work-07b3c8?style=for-the-badge&logo=kaggle)

### Modeling in Progress ....... 